In [45]:
from cw.io import read_cw_data, read_efas_data, read_cw_eu_station
from cw.cfg import DATA_DIR
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

In [46]:
cw_data = read_cw_data()
print(f"Total number of readings are : {len(cw_data)}")
print(f"Total number of stations are : {cw_data.ROOT_ID.nunique()}")

Total number of readings are : 34415
Total number of stations are : 5999


/home/esowc31/CW4Floods/cw/io.py:45: DtypeWarning: Columns (2,3,7,11,16,28,36,38,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_name)


In [47]:
keep = ['ROOT_ID', 'LATITUDE', 'LONGITUDE', 'WATER_LEVEL', 'SPOTTED_AT']
cw_data = cw_data.filter(keep)
cw_data

,ROOT_ID,LATITUDE,LONGITUDE,WATER_LEVEL,SPOTTED_AT
0,16726,47.029225,9.433322,NaN,2017-02-05 14:49
1,16726,47.029225,9.433322,NaN,2017-02-05 14:50
2,16736,47.398311,8.543917,NaN,2017-02-10 08:36
3,16737,47.560411,7.589407,NaN,2017-02-11 12:36
4,16740,47.12788,8.744036,NaN,2017-02-12 10:39
...,...,...,...,...,...
34410,233176,47.985418,7.959271,minus 3,2022-06-03 18:04
34411,628930,46.30159,7.870097,NaN,2022-06-04 13:07
34412,474150,47.201162,9.311755,NaN,2022-06-04 13:46
34413,402713,47.200459,9.311807,NaN,2022-06-04 13:47


In [48]:
cw_data = cw_data[cw_data.WATER_LEVEL.notna()]
cw_data = cw_data[cw_data.WATER_LEVEL != "false" ]


print(f"Total number of readings are : {cw_data.shape[0]}")
print(f"Total number of stations are : {cw_data.ROOT_ID.nunique()}")

Total number of readings are : 7320
Total number of stations are : 579


In [49]:
def str_to_num(x):
    for i in range(7):
        if x == f"minus {i}":
            return -i
        elif x == f"plus {i}":
            return i
        else :
            pass


cw_data.WATER_LEVEL = cw_data.WATER_LEVEL.map(str_to_num)

cw_data

,ROOT_ID,LATITUDE,LONGITUDE,WATER_LEVEL,SPOTTED_AT
27,16845,47.505968,8.7729,-1.0,2017-03-04 17:03
32,16751,47.480988,8.708543,1.0,2017-03-11 18:17
33,16751,47.480988,8.708543,1.0,2017-03-12 11:52
71,16845,47.505968,8.7729,-2.0,2017-04-05 18:33
92,17372,45.975432,8.890493,1.0,2017-04-26 07:56
...,...,...,...,...,...
34381,301198,47.395523,8.73067,-2.0,2022-06-03 13:12
34397,432793,46.461532,6.837685,-1.0,2022-06-03 17:52
34402,432793,46.461532,6.837685,-1.0,2022-06-04 09:20
34408,484886,46.302163,7.87324,-1.0,2022-06-04 13:04


In [50]:
reading_frequency = np.unique(cw_data.ROOT_ID, return_counts = True)
sorted_freq = sorted(zip(reading_frequency[1], reading_frequency[0]), reverse=True)
tuples = zip(*sorted_freq)
freq, station_id = [ list(tuple) for tuple in tuples]

freq_df = pd.DataFrame({"ROOT_ID" : station_id, "FREQ" : freq})
freq_df

,ROOT_ID,FREQ
0,23445,923
1,42809,561
2,221750,451
3,301198,387
4,21368,292
...,...,...
574,17635,1
575,17229,1
576,17215,1
577,16966,1


In [51]:
index = [i for i, val in enumerate(freq) if val>1]
station_id = [station_id[i] for i in index]
cw_data = cw_data[cw_data.ROOT_ID.isin(station_id)]

cw_data.ROOT_ID = cw_data.ROOT_ID.astype('Int64')

cw_data = cw_data.join(freq_df.set_index("ROOT_ID"), on="ROOT_ID")

print(f"Total number of readings are : {cw_data.shape[0]}")
print(f"Total number of stations are : {cw_data.ROOT_ID.nunique()}")

cw_data

Total number of readings are : 7125
Total number of stations are : 384


/tmp/ipykernel_14875/4119091887.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cw_data.ROOT_ID = cw_data.ROOT_ID.astype('Int64')


,ROOT_ID,LATITUDE,LONGITUDE,WATER_LEVEL,SPOTTED_AT,FREQ
27,16845,47.505968,8.7729,-1.0,2017-03-04 17:03,14
32,16751,47.480988,8.708543,1.0,2017-03-11 18:17,7
33,16751,47.480988,8.708543,1.0,2017-03-12 11:52,7
71,16845,47.505968,8.7729,-2.0,2017-04-05 18:33,14
92,17372,45.975432,8.890493,1.0,2017-04-26 07:56,7
...,...,...,...,...,...,...
34381,301198,47.395523,8.73067,-2.0,2022-06-03 13:12,387
34397,432793,46.461532,6.837685,-1.0,2022-06-03 17:52,87
34402,432793,46.461532,6.837685,-1.0,2022-06-04 09:20,87
34408,484886,46.302163,7.87324,-1.0,2022-06-04 13:04,9


In [63]:

cw_eu_station = read_cw_eu_station()
cw_eu_station.ROOT_ID = cw_eu_station.ROOT_ID.astype('Int64')

In [64]:
cw_data = cw_data.join(cw_eu_station.set_index("ROOT_ID"), on="ROOT_ID")
print(f"Total number of readings are : {cw_data.shape[0]}")
print(f"Total number of stations are : {cw_data.ROOT_ID.nunique()}")

Total number of readings are : 7125
Total number of stations are : 384


In [69]:
cw_data = cw_data[cw_data.Minimum_Distance_KM.notna()]
cw_data = cw_data.sort_values("FREQ", ascending=False)
print(f"Total number of readings are : {cw_data.shape[0]}")
print(f"Total number of stations are : {cw_data.ROOT_ID.nunique()}")
cw_data

Total number of readings are : 5498
Total number of stations are : 247


,ROOT_ID,LATITUDE,LONGITUDE,WATER_LEVEL,SPOTTED_AT,FREQ,Minimum_Distance_KM,Lat_Index,Lon_Index
14307,23445,47.726072,13.065014,3.0,2020-07-20 09:08,923,2.485,552.0,410.0
3923,23445,47.726072,13.065014,6.0,2019-03-16 17:05,923,2.485,552.0,410.0
12742,23445,47.726072,13.065014,3.0,2020-05-15 14:52,923,2.485,552.0,410.0
4005,23445,47.726072,13.065014,2.0,2019-03-21 17:43,923,2.485,552.0,410.0
3985,23445,47.726072,13.065014,1.0,2019-03-20 16:22,923,2.485,552.0,410.0
...,...,...,...,...,...,...,...,...,...
10055,44302,47.075147,9.468846,1.0,2019-12-08 13:57,2,1.867,567.0,356.0
25759,463608,46.480049,9.827422,-1.0,2021-08-13 08:57,2,1.201,580.0,361.0
33269,497846,46.25346,7.416832,-1.0,2022-04-27 10:49,2,2.283,585.0,324.0
3252,62901,47.986228,7.947854,-2.0,2018-12-25 15:09,2,1.765,546.0,333.0


In [70]:
fig, ax = plt.subplots()
ax.hist(cw_data.SPOTTED_AT, bins = 10)
plt.show()

(array([550., 545., 548., 550., 550., 557., 547., 548., 551., 552.]),
 array([   0. ,  545.5, 1091. , 1636.5, 2182. , 2727.5, 3273. , 3818.5,
        4364. , 4909.5, 5455. ]),
 <BarContainer object of 10 artists>)